In [ ]:
from s3prl.downstream.mdd.dataset import L2ArcticDataset
from s3prl.downstream.specaug import SpecAug
from s3prl.downstream.augment_utils.spectrogram import Spectrogram
from IPython.display import Audio, display
from torch.utils.data import RandomSampler, DataLoader, Subset
import torch
import einops


In [ ]:
dataset = L2ArcticDataset(
    'train',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/'
)

In [ ]:
def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()
  display(Audio(waveform, rate=sample_rate))

In [ ]:
play_audio(dataset[0][0], 44100)

In [ ]:
SAMPLE_RATE = 44100
n_fft = 512
n_stft = int((n_fft // 2) + 1)
specaug = SpecAug(
    **{
        "adaptive": False,
        "adaptive_number_ratio": 0.04,
        "adaptive_size_ratio": 0.04,
        "max_n_time_masks": 20,
        "apply_time_warp": True,
        "apply_time_mask": True,
        "apply_freq_mask": True,
        "time_warp_window": 5,
        "time_mask_width_range": [0, 40],
        "freq_mask_width_range": [0, 50],
        "num_freq_mask": 4,
        "num_time_mask": 2,
    }
)

device = 'cuda'

transform = Spectrogram(num_stft_frequencies=n_fft)
num_samples=10
subset = Subset(dataset, torch.arange(num_samples))
wavs = [data[0] for data in subset]
aug_specs = [specaug(einops.rearrange(transform.wav_to_spec(wav), "h w complex -> complex h w"))[0] for wav in wavs]
aug_wavs = [transform.spec_to_wav(torch.complex(specs[0], specs[1])) for specs in aug_specs]
play_audio(aug_wavs[3].cpu(), 44100)

In [ ]:
SAMPLE_RATE = 44100
n_fft = 2048
n_stft = int((n_fft // 2) + 1)
specaug = SpecAug(
    **{
        "adaptive": False,
        "adaptive_number_ratio": 0.04,
        "adaptive_size_ratio": 0.04,
        "max_n_time_masks": 20,
        "apply_time_warp": True,
        "apply_time_mask": True,
        "apply_freq_mask": True,
        "time_warp_window": 5,
        "time_mask_width_range": [0, 40],
        "freq_mask_width_range": [0, 5],
        "num_freq_mask": 4,
        "num_time_mask": 2,
    }
)

device = "cuda"

transform = Spectrogram(num_stft_frequencies=n_fft)
spectrogram = transform.wav_to_spec(dataset[0][0])
print(spectrogram.size())
augmented = [specaug([spectrogram[:, :, 0]])[0][0], specaug([spectrogram[:, :, 1]])[0][0]]
display(augmented[0][0].size())
display(augmented[0])
raw = transform.spec_to_wav(torch.complex(augmented[0], augmented[1]))

play_audio(raw, 44100)